# From raw *.ome.tif file to kinetic properties for immobile particles
This notebook will run ... 
* picasso_addon.localize.main()
* picasso_addon.autopick.main()
* spt.immobile_props.main()

... in a single run to get from the raw data to the fully evaluated data in a single stroke. We therefore: 

1. Define the full paths to the *ome.tif files
2. Set the execution parameters
3. Connect or start a local dask parallel computing cluster
4. Run all sub-module main() functions for all defined datasets

As a result files with extension *_locs.hdf5, *_render.hdf5, *_autopick.yaml, *_tprops.hdf5 will be created in the same folder as the *.ome.tif file.

In [ ]:
import os
import traceback
import importlib
from dask.distributed import Client
import multiprocessing as mp

import picasso.io as io
import picasso_addon.localize as localize
import picasso_addon.autopick as autopick
import spt.immobile_props as improps

importlib.reload(localize)
importlib.reload(autopick)
importlib.reload(improps)

### 1. Define the full paths to the *ome.tif files

In [ ]:
dir_names=[]
dir_names.extend([r'C:\Data\p06.SP-tracking\20-03-11_pseries_fix_B21_rep\id140_B_exp200_p114uW_T21_1\test'])

file_names=[]
file_names.extend(['id140_B_exp200_p114uW_T21_1_MMStack_Pos0.ome.tif'])

### 2. Set the execution parameters

In [ ]:
### Valid for all evaluations
params_all={'undrift':False,
            'min_n_locs':5,
            'filter':'fix',
            }

### Exceptions
params_special={}

All possible parameters for ...
* picasso_addon.localize.main()
* picasso_addon.autopick.main()
* spt.immobile_props.main() 

... can be given. Please run `help(localize.main)` or `help(autopick.main)` or `help(improps.main)` or readthedocs. If not stated otherwise standard values are used (indicated in brackets).

In [5]:
help(localize.main)

Help on function main in module picasso_addon.localize:

main(file, info, path, **params)
    Localize movie (least squares, GPU fitting if available) and undrift resulting localizations using rcc.
     
    Args:
        file(picasso.io):          Either raw movie loaded with picasso.io.load_movie() or_locs.hdf5 loaded with picasso.io.load_locs()
        info(list(dicts)):         Info to raw movie/_locs.hdf5 loaded with picasso.io.load_movie() or picasso.io.load_locs()
    
    Keyword Arguments:
        localize(bool=True)        Localize raw movie (see picasso.localize)
        baseline(int=70):          Camera spec. baseline (see picasso.localize).
        gain(float=1):             Camera spec. EM gain (see picasso.localize)
        sensitivity(float=0.56):   Camera spec. sensitivity (see picasso.localize)
        qe(float=0.82):            Camera spec. sensitivity (see picasso.localize)
        box(int=5):                Box length (uneven!) of fitted spots (see picasso.localize

### 3. Connect or start a local dask parallel computing cluster
This is only necessary if you want to use parallel computing for the spt.immobile.props.main() execution (standard). If not set `params_all={'parallel':False}`

In [7]:
try:
    client = Client('localhost:8787')
    print('Connecting to existing cluster...')
except OSError:
    improps.cluster_setup_howto()

Connecting to existing cluster...


If we execute the prompt (see below) a local cluster is started, and we only have to execute the cell above  to reconnect to it the next time. If you try to create a new cluster under the same address this will throw an error!

In [9]:
Client(n_workers=max(1,int(0.8 * mp.cpu_count())),
       processes=True,
       threads_per_worker=1,
       scheduler_port=8787,
       dashboard_address=":1234")

KeyError: 'scheduler_port'

### 4. Run all sub-module main() functions for all defined datasets

In [ ]:
failed_path=[]
for i in range(0,len(file_names)):
    ### Create path
    path=os.path.join(dir_names[i],file_names[i])
    
    ### Set paramters for each run
    params=params_all.copy()
    for key, value in params_special.items():
        params[key]=value[i]
        
    ### Run main function
    try:
        ### Load movie
        movie,info=io.load_movie(path)
        
        ### Localize and undrift
        out=localize.main(movie,info,path,**params)
        info=info+[out[0][0]]+[out[0][1]] # Update info to used params
        path=out[-1] # Update path
        
        ### Autopick
        print()
        locs=out[1]
        out=autopick.main(locs,info,path,**params)
        info=info+[out[0]] # Update info to used params
        path=out[-1] # Update path
        
        ### Immobile kinetics analysis
        print()
        locs=out[1]
        out=improps.main(locs,info,path,**params)
        
    except Exception:
        traceback.print_exc()
        failed_path.extend([path])

print()    
print('Failed attempts: %i'%(len(failed_path)))